In [17]:
!pip install openai==0.28

In [27]:
import os
import pandas as pd
import openai
import re
import json  # To handle reviews_list parsing

# Load and preprocess the data
df = pd.read_csv('zomato.csv')  # Make sure to use the correct path

# Preprocessing steps
df.drop_duplicates(subset=['name', 'address', 'cuisines'], inplace=True)
df['rate'] = pd.to_numeric(df['rate'].apply(lambda x: x.split('/')[0] if isinstance(x, str) else None), errors='coerce')
df['rate'].fillna(df['rate'].mean(), inplace=True)
df['approx_cost(for two people)'] = pd.to_numeric(df['approx_cost(for two people)'].str.replace(',', ''), errors='coerce')

# Attempt to convert reviews_list from string to list if it's not already a list
def parse_reviews(reviews_str):
    try:
        reviews = json.loads(reviews_str.replace("\'", "\""))
        return ", ".join([rev['review_text'] for rev in reviews[:1]])  # Get first review text as a sample
    except:
        return "No reviews available"

df['reviews_list'] = df['reviews_list'].apply(parse_reviews)

# Securely set your OpenAI API key from an environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')
def get_chat_model_completions(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            max_tokens=100
        )
        # Adjusting to the expected structure of ChatCompletion responses
        return response['choices'][0]['message']['content'].strip()
    except openai.error.InvalidRequestError as e:
        print(f"An error occurred: {str(e)}")
    except Exception as e:
        print(f"Unexpected error: {str(e)}")


def parse_intent(intent):
    cuisine_match = re.search(r'(\w+) restaurant', intent)
    location_match = re.search(r'in (\w+)', intent)
    rate_match = re.search(r'rate of (\d+(\.\d+)?)', intent)
    
    cuisine = cuisine_match.group(1) if cuisine_match else "Italian"
    location = location_match.group(1) if location_match else "Banashankari"
    rate = float(rate_match.group(1)) if rate_match else 4.0
    
    return cuisine, location, rate

def recommend_restaurants(cuisine, location, minimum_rate=4.0):
    filtered_df = df[(df['cuisines'].str.contains(cuisine, case=False, na=False)) &
                     (df['location'].str.contains(location, case=False, na=False)) &
                     (df['rate'] >= minimum_rate)]
    
    recommendations = filtered_df.sort_values(by='rate', ascending=False).head(5)
    return recommendations

def format_recommendations(recommendations):
    if recommendations.empty:
        return "No recommendations could be found based on your criteria."
    
    formatted = "Top Restaurant Recommendations:\n"
    for idx, row in recommendations.iterrows():
        formatted += f"{idx+1}. {row['name']} - Cuisine: {row['cuisines']} - Rating: {row['rate']}\n"
        formatted += f"    Type: {row['rest_type']}, Liked Dishes: {row['dish_liked']}, Cost for Two: INR {row['approx_cost(for two people)']}\n"
        formatted += f"    Review Sample: {row['reviews_list']}\n"
    return formatted

def process_user_query(query):
    intent = get_chat_model_completions(query)
    if not intent or "An error occurred:" in intent:
        return "Failed to process query due to an error."
    
    cuisine, location, minimum_rate = parse_intent(intent)
    recommendations = recommend_restaurants(cuisine, location, minimum_rate)
    return format_recommendations(recommendations)

# Example usage
user_query = "I'm looking for an Italian restaurant in Banashankari with a rate of 4."
recommendations = process_user_query(user_query)
print(recommendations)


Top Restaurant Recommendations:
8. Onesta - Cuisine: Pizza, Cafe, Italian - Rating: 4.6
    Type: Casual Dining, Cafe, Liked Dishes: Farmhouse Pizza, Chocolate Banana, Virgin Mojito, Pasta, Paneer Tikka, Lime Soda, Prawn Pizza, Cost for Two: INR 600.0
    Review Sample: No reviews available
10. Smacznego - Cuisine: Cafe, Mexican, Italian, Momos, Beverages - Rating: 4.2
    Type: Cafe, Liked Dishes: Waffles, Pasta, Coleslaw Sandwich, Choco Waffle, Tacos, Momos, Cheese Nachos, Cost for Two: INR 550.0
    Review Sample: No reviews available
12. Cafe Shuffle - Cuisine: Cafe, Italian, Continental - Rating: 4.2
    Type: Cafe, Liked Dishes: Mocktails, Peri Fries, Lasagne, Pizza, Chicken Bbq Wings, Virgin Mojito, Nachos, Cost for Two: INR 600.0
    Review Sample: No reviews available
13. The Coffee Shack - Cuisine: Cafe, Chinese, Continental, Italian - Rating: 4.2
    Type: Cafe, Liked Dishes: Coffee, Spaghetti, Pancakes, Nachos, Pasta, Sandwich, Garlic Bread, Cost for Two: INR 500.0
    Revi